<a href="https://colab.research.google.com/github/SlickMik/ensf692Project/blob/main/ENSF_692_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ESNF 692 Project
### Group # : 4
##### Authors : Danish Shahid () | Muhammad Ibrahim Khan ()
---

This notebook describes the correlation of asset classes - Oil, Gold, and S%P500 for a user selected time range

---

Importing Libraries

In [ ]:
import kagglehub
import numpy as np
import pandas as pd
import matplotlib as mp

# Dataset Selection

Downloading Kaggle Datasets

In [ ]:
# Download latest dataset versions
gold_dataset_path = kagglehub.dataset_download("ritwikb3/daily-gold-price-1996-2023-time-series")

oil_dataset_path = kagglehub.dataset_download("tarique7/daily-crude-price-dataset")

sp_dataset_path = kagglehub.dataset_download("paveljurke/s-and-p-500-gspc-historical-data")

# Dataset Creation

Reading Dataset to Pandas

In [ ]:
# read csv files into pandas
gold_dataset = pd.read_csv(gold_dataset_path + f'/deccan gold mines ltd eod price 2.csv')

oil_dataset = pd.read_csv(oil_dataset_path + f'/Crude Oil WTI Futures Historical Data.csv')

sp_dataset = pd.read_csv(sp_dataset_path + f'/sap500.csv')

Standardizing Date columns to Timestamps

In [ ]:
gold_dataset['Date'] = pd.to_datetime(gold_dataset['Date'])
oil_dataset['Date'] = pd.to_datetime(oil_dataset['Date'])
sp_dataset['Date'] = pd.to_datetime(sp_dataset['Date'])

Set indices to Column 'Date'

In [ ]:
gold_dataset = gold_dataset.set_index('Date')
oil_dataset = oil_dataset.set_index('Date')
sp_dataset = sp_dataset.set_index('Date')

Brain storming

In [ ]:
assets_df = gold_dataset.join(oil_dataset, how='outer', lsuffix='_Gold', rsuffix='_WTI').join(sp_dataset, how='outer', rsuffix='_SP')

In [ ]:
gold_dataset.loc['1996':'1998']

KeyError: 'Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is not allowed.'

In [ ]:
assets_df['1996-12-30':'1997-12-30'].head(10)

,Open_Gold,High_Gold,Low_Gold,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,...,Open_WTI,High_WTI,Low_WTI,Vol.,Change %,Open,High,Low,Close_SP,Volume
Date,,,,,,,,,,,,,,,,,,,,,
1996-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.17,25.55,25.12,22.35K,0.59%,756.789978,759.200012,752.729980,753.849976,339060000.0
1996-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.29,25.97,25.22,21.48K,2.17%,753.849976,753.950012,740.739990,740.739990,399760000.0
1997-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.88,26.05,25.60,29.31K,-0.89%,740.739990,742.809998,729.549988,737.010010,463230000.0
1997-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.72,25.78,25.35,25.04K,-0.39%,737.010010,748.239990,737.010010,748.030029,452970000.0
1997-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.70,26.47,25.63,43.20K,3.05%,748.030029,753.309998,743.820007,747.650024,531350000.0
1997-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26.27,26.45,26.07,36.55K,-0.53%,747.650024,753.260010,742.179993,753.229980,538220000.0
1997-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26.20,26.65,25.85,50.76K,1.49%,753.229980,755.719971,747.710022,748.409973,557510000.0
1997-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26.60,26.74,26.25,43.25K,-0.94%,748.409973,757.679993,748.409973,754.849976,555370000.0
1997-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26.32,26.41,26.00,41.90K,-1.06%,754.849976,759.650024,746.919983,759.500000,545850000.0


In [ ]:
combine_df

,0
Date_Gold,4846
Open_Gold,4846
High_Gold,4846
Low_Gold,4846
Close,4846
WAP,4846
No. of Shares,4846
No. of Trades,4846
Total Turnover,4846
Deliverable Quantity,4702


# User Entry

In [ ]:
ra = input('please enter your input')

please enter your inputhello


# Analysis & Calculations

In [ ]:
# Analysis and Calculations

# Export & Matplotlib